# 参数管理

我们首先看一下具有单隐藏层的多层感知机

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3591],
        [-0.3060]], grad_fn=<AddmmBackward0>)

参数访问

In [2]:
# 状态字典
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1460, -0.0880, -0.0841, -0.3279, -0.2297,  0.2477, -0.1056,  0.3343]])), ('bias', tensor([-0.2009]))])


目标参数

weight 权重  
可以看到此处有 8 个参数

bias 偏移  
在神经网络中，偏置项（bias）是指在神经元中的一个常数，它与输入相乘后加到激活函数的输入中。 在 PyTorch 中，偏置项是线性层的参数之一，它是一个向量，其长度等于该层的输出大小。  
在你的代码中，net[2].bias.data 返回的是第三个线性层的偏置项。因此，你可以认为它是 linear 公式的参数 b 的值。

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0291], requires_grad=True)
tensor([-0.0291])


In [4]:
net[2].weight.grad == None

True

一次性访问所有参数
> 中间的 ReLU 是没有的

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
net.state_dict()['2.bias'].data

tensor([-0.0291])

从嵌套块收集参数

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3078],
        [-0.3078]], grad_fn=<AddmmBackward0>)

设计了网络后，我们看看它是如何工作的

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [9]:
rgnet[0][1][0].bias.data

tensor([-0.2539,  0.4913,  0.3029, -0.4799,  0.2022,  0.3146,  0.0601,  0.3757])

内置初始化

全连接层（Fully Connected Layer）是指神经网络中的一种常见层类型，也称为线性层（Linear Layer）。在全连接层中，每个输入神经元都与输出神经元相连，因此该层的输出是输入的线性组合。全连接层通常用于将高维输入数据映射到低维空间，或者将低维特征映射到高维空间。

In [10]:
# m 是一个 module 
def init_normal(m):
    # 判断是否是全连接层
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
# 应用到 net 中的每一个全连接层
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0128, -0.0141,  0.0062,  0.0028]), tensor(0.))

In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应用不同的初始化方法

In [12]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.3809,  0.5354, -0.4686, -0.2376])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, 0.0000, -0.0000, 0.0000],
        [-0.0000, 9.3464, 5.5061, 6.8197]], grad_fn=<SliceBackward0>)

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

参数绑定

In [15]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
